In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.tree import DecisionTreeClassifier

from src.evaluate_regression import get_rankings, average_spearman

# Load Data

In [51]:
FACTORS = ["dataset", "model", "tuning", "scoring"]
NEW_INDEX = "encoder"

In [52]:
# Load Train Data
X_train = pd.read_csv("../../data/preprocessed/X_train.csv")
y_train = pd.read_csv("../../data/preprocessed/y_train.csv")

# Load holdout data
X_holdout_original = pd.read_csv("../../data/preprocessed/X_hold_out_original.csv")
X_holdout = pd.read_csv("../../data/preprocessed/X_holdout.csv")
y_holdout = pd.read_csv("../../data/preprocessed/y_holdout.csv")

print("Shape of X_train ", X_train.shape)
print("Shape of y_train ", y_train.shape)
print(50 * "-")
print("Shape of X_holdout_original ", X_holdout_original.shape)
print("Shape of X_holdout ", X_holdout.shape)
print("Shape of y_holdout ", y_holdout.shape)

X_train.head(5)

Shape of X_train  (26989, 112)
Shape of y_train  (26989, 1)
--------------------------------------------------
Shape of X_holdout_original  (9065, 5)
Shape of X_holdout  (9065, 112)
Shape of y_holdout  (9065, 1)


,dataset,model_DTC,model_KNC,model_LGBMC,model_LR,model_SVC,tuning_full,tuning_model,tuning_no,scoring_ACC,...,non_categorical_features_count,ratio_of_categorical_features_to_all,sum_of_all_categories,categorical_target_variables_count,non_categorical_target_variables_count,categorical_target_values_sum,total_feature_count,min_number_of_categories_per_cat_feature,max_number_of_categories_per_cat_feature,avg_number_of_categories_per_cat_feature
0,0.025297,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.564327,0.045059,1.0,0.0,0.0,0.0,0.476695,0.0,0.705853,0.429681
1,0.025297,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.564327,0.045059,1.0,0.0,0.0,0.0,0.476695,0.0,0.705853,0.429681
2,0.025297,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.564327,0.045059,1.0,0.0,0.0,0.0,0.476695,0.0,0.705853,0.429681
3,0.025297,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.564327,0.045059,1.0,0.0,0.0,0.0,0.476695,0.0,0.705853,0.429681
4,0.025297,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.564327,0.045059,1.0,0.0,0.0,0.0,0.476695,0.0,0.705853,0.429681


# Prepare for ordinal regression

In [53]:
y_train.head(5)

,rank
0,21.0
1,19.0
2,26.0
3,12.0
4,28.0


In [54]:
def transform_y_train_to_multi_output_binary_classification(y_train: pd.DataFrame, verbosity:int =0) -> pd.DataFrame:
    """
    Transforms y_train to the schema that a multi output binary classification problem expects.
    Transform rank into 31 binary targets, y_i’s: y_i = 1 if rank > i, 0 otherwise
    Note that y_i = 1 implies y_{i-1} = 1.

    :param y_train: pd.DataFrame -- Dataframe of the labels of the train set
    :param verbosity: int -- Level of verbosity

    :return: y_train: pd.DataFrame -- Transformed dataframe
    """
    y_train = y_train.copy()

    if verbosity > 0:
        print("Transforming y_train to multi output binary classifcation ...")

    # y_i’s: y_i = 1 if rank > i, 0 otherwise
    for i in range(int(y_train["rank"].max())):
        y_train[f"y{i}"] = (y_train["rank"] > i).astype(int)
    # Drop original target
    y_train.drop("rank", axis=1, inplace=True)

    return y_train

y_train = transform_y_train_to_multi_output_binary_classification(y_train=y_train)
y_train.head(5)

,y0,y1,y2,y3,y4,y5,y6,y7,y8,y9,...,y21,y22,y23,y24,y25,y26,y27,y28,y29,y30
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,0,0,0
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,0,0,0


# Modelling

In [55]:
# Fit model
model = DecisionTreeClassifier(random_state=42).fit(X_train, y_train)

# Predict on holdout set
y_pred = model.predict(X_holdout)

In [59]:
y_pred

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

# Scoring

In [63]:
def score_model(X_holdout_original: pd.DataFrame, y_holdout, y_pred, verbosity = 1):
    """
    Scores the prediction set y_pred using the provided functions from supervisor.

    :param X_holdout_original: pd.DataFrame -- Schema of the original holdout set before any processing
    :param y_holdout: pd.DataFrame -- Dataframe of the labels of the holdout set
    :param y_pred: Vector -- Vector of the predicted labels of the holdout set X_holdout
    :param verbosity: int -- Level of verbosity

    :return: average_spearman_sum, average_spearman_min
    """
    X_holdout_original["rank"] = y_holdout
    X_holdout_original["rank_pred_sum"] = y_pred.sum(axis=1)
    X_holdout_original["rank_pred_min"] = np.argmin(y_pred, axis=1) - 1

    rankings_test = get_rankings(X_holdout_original, factors=FACTORS, new_index=NEW_INDEX, target="rank")
    rankings_pred_sum = get_rankings(X_holdout_original, factors=FACTORS, new_index=NEW_INDEX, target="rank_pred_sum")
    rankings_pred_min = get_rankings(X_holdout_original, factors=FACTORS, new_index=NEW_INDEX, target="rank_pred_min")

    average_spearman_sum = average_spearman(rankings_test, rankings_pred_sum)
    average_spearman_min = average_spearman(rankings_test, rankings_pred_min)

    if verbosity > 0:
        print("Ordinal regression average spearman:\n",
            f"Sum: {average_spearman_sum:.4f}\n",
            f"Min: {average_spearman_min:.4f}")

    return average_spearman_sum, average_spearman_min


average_spearman_sum, average_spearman_min = score_model(X_holdout_original, y_holdout, y_pred, verbosity=1)

Ordinal regression average spearman:
 Sum: 0.6350
 Min: 0.6170
